# Segment Tree

## Introducción

Al momento de utilizar datos estáticos, uno suele realizar operaciones basadas en ellos con alta flexibilidad debido a que estos no varían, dando respuestas fijas a consultas fijas realizadas sobre los mismos.

Sin embargo, en el mundo real, los datos suelen variar con el tiempo en su mayoría, por lo que necesitamos alguna forma de poder manipularlos eficientemente con sus modificaciones y nuevas respuestas a las consultas hechas.

Una estructura de datos muy flexible respecto a las operaciones que se pueden realizar sobre los datos es el **Segment Tree**, que permite actualizar y consultar en un tiempo suficientemente rápido en la mayoría de casos.

## Idea base - Segment Tree Simple

El Segment Tree puede ser analizado tomando una función $f$ sobre los datos a analizar (posteriormente daremos las características necesarias para que la función pueda funcionar correctamente) y su punto de partida es similar al del algoritmo de Merge Sort: Divide & Conquer.

### Construcción

Supongamos que tenemos $n$ elementos $a_{i}$, con $i=1,\ldots,n$; entonces nuestra idea será usar un árbol binario, de forma que en cada nodo mantendremos información de un rango continuo de posiciones de nuestros elementos. Además de que un nodo $(l,r)$ guarde la información en el rango $[l,r]$, este tendrá 2 hijos **si y solo si** $l \neq r$, los cuales serán:

$$ \begin{array}{cc} \text{Hijo Izquierdo}: &\left(l,\left\lfloor\frac{l+r}{2}\right\rfloor\right) \\ \text{Hijo Derecho}: &\left(\left\lfloor\frac{l+r}{2}\right\rfloor+1,r\right) \end{array} $$

Ahora, esta idea funciona correctamente para elementos estáticos; sin embargo, para que nuestro enfoque tenga sentido, necesitamos el **Conquer**, así que una condición que tendremos será que **La información a almacenar de un nodo depende de la información guardada en sus hijos (si es que tiene) o es un dato puntual**.

La anterior condición se reformula en dos partes:

1) La información de (l,r), con $l \neq r$ depende de la información de sus hijos definidos anteriormente.

2) La información de (l,r) con $l = r$ es un dato puntual (lo cual tiene sentido puesto que es un solo elemento del rango de datos).

Debido a la condición 1, notamos que $Info(p) = f(Info(l),Info(r))$ para un nodo $p$ con hijos $l$ y $r$ (izquierdo y derecho respectivamente), lo cual implica que:

$$ Info(l,r) = f\left(Info\left(l,\left\lfloor\frac{l+r}{2}\right\rfloor\right),Info\left(\left\lfloor\frac{l+r}{2}\right\rfloor+1,r\right)\right) $$

Ahora la plantilla de implementación también estará basada en el Merge Sort, tomando una función `build` que construya los nodos de menor tamaño para luego combinarlos con otra función `merge` y obtener la respuesta de los nodos de mayor tamaño.

```Python
build(pos,l,r):
    if l == r:
        st[pos] = getInfo(a[l])
        return
    mi = (l+r)/2
    build(L[pos],l,mi)
    build(R[pos],mi+1,r)
    st[pos] = merge(st[L[pos]],st[R[pos]])
```

Donde `st[pos]` es una estructura adecuada para mantener la información, así como la función `getInfo` es autoexplicativa. Por último, notemos que usamos notación como en un arbol binario almacenado en un arreglo, por lo que el hijo izquierdo y el derecho de cada nodo estarán en un arreglo extra cada uno, llamados `L` y `R` respectivamente.

Finalmente, como el arbol tiene $n$ hojas (debido a que los únicos que no tienen hijos, por definición, son los rangos de tamaño 1), entonces la **cantidad de nodos** en total será menor que $2n$, por lo tanto la construcción será de $O(n)$. Asimismo, el árbol tendrá una altura $O(\log{n})$.

**Nota:** Por ahora, $f$ no tiene ninguna característica obligatoria.
**Nota 2:** Nótese que la cantidad de nodos puede ser menor que $2n$ pero algunas implementaciones pueden tomar $4n$ de memoria para los nodos.

### Consultas

Ahora, para poder responder consultas como si fueran datos estáticos, necesitamos una manera de relacionar la información que tenemos en el árbol para satisfacerlas de manera eficiente. Para lo anterior, nos encontramos con dos posibles situaciones:

1) El rango es exactamente el usado en algún nodo del árbol: La respuesta se dará de manera inmediata una vez visitado el nodo correspondiente.

2) El rango es una mezcla de nodos no relacionados de manera directa (uno no es padre alguno otro): Necesitaremos plantear una condición sobre $f$ para que nuestro árbol no haya sido calculado en vano, la cual es que **$f$ sea asociativa**.

Nuestra función de consulta considerará lo siguiente al analizar el nodo $pos$ que cubre el rango $(l,r)$:

1) Si es que $[l,r]$ está dentro del rango que se desea consultar, entonces devolvemos Info(pos).

2) Si es que $[l,r]$ no se intersecta con el rango que se desea consultar, entonces devolvemos el elemento neutro de $f$.

3) Si no se da 1 ni 2, entonces realizamos las consultas sobre los hijos de $pos$ y las mezclamos para dar la respuesta del rango actual. Esta acción da la información correcta debido a 2, pues los elementos que debemos ignorar se manifestarán como el elemento neutro de $f$ y no afectarán a la respuesta.

Aprovechando los casos anteriores podemos responder las consultas en $O(\log{n})$, considerando que en cada nivel del árbol se **expanden** (realizar una recursión sobre los hijos) a lo mucho 2 nodos, y dado que la altura del árbol es $O(\log{n})$, se concluye la proposición inicial.

Nuestra plantilla de consulta será una ligera variación usando los 3 casos anteriores:

```Python
query(x,y,pos,l,r):
    if y < l or r < x: return NIL # Elemento neutro porque no se intersectan
    if x <= l and r <= y: return st[pos] # Incluido por completo en el rango
    mi = (l+r)/2
    L = query(x,y,L[pos],l,mi) # Hijo izquierdo
    R = query(x,y,R[pos],mi+1,r) # Hijo derecho
    return merge(L,R) # Devolvemos la mezcla de las informaciones
```

### Actualizaciones Puntuales

Para modificar **un solo elemento** de los datos, notamos que solamente la hoja con el rango $(l,l)$ tendrá su información, y si queremos modificar ese nodo, entonces existe un solo camino desde la raíz hasta este. Además de lo anterior, solamente los nodos que sean visitados por ese camino serán modificados como consecuencia, por lo que la actualización modificará $O(\log{n})$ nodos, dándonos una complejidad de $O(\log{n})$.

Esta función es casi igual a la de construcción, solo que descarta uno de los hijos (el que no está en el camino) y reconstruye la información cada vez que se pasa por algún ancestro.

```Python
update(x,y,pos,l,r):
    if l == r:
        st[pos] = getInfo(y)
        return
    mi = (l+r)/2
    if l <= x and x <= mi: # El elemento esta en el subarbol del hijo izquierdo
        update(x,y,L[pos],l,mi)
    else:
        update(x,y,R[pos],mi+1,r) # El elemento esta en el subarbol del hijo derecho
    st[pos] = merge(st[L[pos]],st[R[pos]])
```

Con todas las consideraciones anteriores sobre la construcción del Segment Tree, tenemos una estructura muy potente, que puede realizar operaciones de manera flexible en $O(\log{n})\cdot C(\text{operacion})$

## Aplicaciones

### Suma en rango

Para mantener una suma en rango, basta con realizar las siguientes consideraciones:

1) `st` es un arreglo adecuado para mantener la suma de los elementos que cubre su rango

2) `merge(a,b)` debe ser reemplazado por `a+b`

3) `getInfo(a)` debe ser reemplazado por `a`

4) `NIL` debe ser reemplazado por 0

### RMQ en rango

Para mantener un RMQ en rango, basta con realizar las siguientes consideraciones:

1) `st` es un arreglo adecuado para mantener la suma de los elementos que cubre su rango

2) `merge(a,b)` debe ser reemplazado por `RMQ(a,b)`

3) `getInfo(a)` debe ser reemplazado por `a`

4) `NIL` debe ser reemplazado por el elemento neutro del RMQ (`-inf` para maximo y `inf` para minimo)

## Ejemplos:

- [XOR](https://codeforces.com/gym/100739/problem/A)
- [Cambio de funcion por nivel](https://codeforces.com/contest/339/problem/D)
- [Parentheses Balance with updates](https://codeforces.com/contest/380/problem/C)
- [Composición de funciones](https://codeforces.com/contest/911/problem/G)
- [ST + DP](https://codeforces.com/contest/597/problem/C)
- [ST + Preprocessing + Offline](https://codeforces.com/contest/459/problem/D)
- [ST + Preprocessing](https://codeforces.com/contest/61/problem/E)
- [Cambio del enfoque de modificacion puntual](https://codeforces.com/contest/356/problem/A)
- [Cambio del enfoque de modificacion puntual + preprocessing](https://codeforces.com/contest/920/problem/F)
- [Cambio de la estructura del nodo](www.spoj.pl/problems/GSS1/)
- [Order Statistic ST + Offline](https://www.spoj.com/problems/ORDERSET/)
